# Data Preparation 

Show and explain all steps involved in your assignment – Data Import, Data Cleaning, Data Preview, Data Description. Must have flow chart.


## First view of the data

### Loading Data

In [ ]:
import pandas as pd
import numpy as np
from pprint import pprint
data_file_path = ""
csv_file_name = "Austin Bicycle Crashes 2010-2017.csv"
dataset = pd.read_csv(data_file_path+csv_file_name)
dataset.head()

In [ ]:
import missingno as msno
msno.matrix(dataset)

In [ ]:
dataset.count().max()

In [ ]:
dataset.dtypes

### Store and Restore the Fields Grouping

In [ ]:
# store the fields in another csv file
data_file_path = ""
csv_file_name = "Austin Bicycle Crashes 2010-2017.csv"

def save_df_csv(csv_str, file_name, path=""):
	f = open(path+file_name, "w")
	f.write(csv_str)
	f.close()
li = list(dataset.columns)
df = pd.DataFrame(li, columns=["fields"])
save_df_csv(df.to_csv(index=False, line_terminator="\n"), "fields before grouping.csv")

In [ ]:
file_path = "data fields grouping.csv"
super_grp = pd.read_csv(file_path)
super_grp = {grp_name: super_grp[grp_name].dropna() for grp_name in super_grp}
# check
from functools import reduce
grp_set = reduce(lambda acm, cur: acm.union(cur), [super_grp[n] for n in super_grp], set())
assert(set([n for n in dataset]).difference(grp_set) == set())
pprint(super_grp)

### Example Using Super Grouping (stub)

In [ ]:
def temp_super_grp_print(): 
	[i for i in super_grp]
	dataset.loc[:, list(super_grp["crash_report"])].head()
	dataset.loc[:, list(super_grp["geo_info"])].head()
	dataset.loc[:, list(super_grp["day"])]
	dataset.loc[:, list(super_grp["person"])]
	dataset.loc[:, list(super_grp["road_cond"])]

### Checking Data
Note that: it is easier to view the data superficially using Excel

The data are revised by going through each 10 of all variables

In [ ]:
# uni abbrv. one
uni_col = list(filter(lambda c: len(dataset[c].unique()) == 1, dataset))
uni_dict = {k:dataset[k].unique()[0] for k in uni_col}
uni_dict

In [ ]:
# remove the column that has only one unique data
dataset.drop([col for col in uni_dict],axis = 1, inplace=True)

In [ ]:
dataset.replace(["No Data"], pd.NA, inplace=True)

Remark: There is a field `street number` use `"NO DATA"` and might be up to a interpretation that where `street number` is no applicable to every street. Hence, we keep it as it is, then evaluated later on demand.

In [ ]:
# see there are how many variables
len(list(dataset.dtypes))

In [ ]:
def leg_stepping(start, end,step = 1):
	'''
		eg.: it = leg_stepping(1, 10, 3)
		print(next(it)) -> (1, 4)
		print(next(it)) -> (5, 8)
		print(next(it)) -> (9, 10)
	'''
	assert(start < end)
	while start+step < end:
		yield {"start": start, "end": start+step}
		start += step+1
	yield {"start": start, "end": end}
ind_iter = leg_stepping(0, len(list(dataset.dtypes)), 10)
tmp_viewer = lambda curr: dataset.iloc[:, curr["start"]:curr["end"]]

In [ ]:
curr_slice = next(ind_iter)
tmp_viewer(curr_slice).head()

In [ ]:
li = ["$1000 Damage to Any One Person's Property", "Active School Zone Flag", "Construction Zone Flag"]
{c:dataset[c].unique() for c in li} # check if binary

In [ ]:
def booleanize(s, true_corrpd, false_corrpd):
	s = s.replace(true_corrpd, True)
	s = s.replace(false_corrpd, False)
	return s

In [ ]:
# changing the yes/no to boolean values
li = ["$1000 Damage to Any One Person's Property", "Active School Zone Flag", "Construction Zone Flag"]
dataset.loc[:,li] = booleanize(dataset.loc[:,li], "Yes", "No")

In [ ]:
curr_slice = next(ind_iter)
tmp_viewer(curr_slice)

In [ ]:
import datetime
converter = lambda n: datetime.time(hour = int(n/100), minute = n%100)
dataset['Crash Time'] = dataset['Crash Time'].apply(converter)

In [ ]:
tmp_viewer(curr_slice).dtypes

In [ ]:
curr_slice = next(ind_iter)
tmp_viewer(curr_slice).head()

In [ ]:
tmp_viewer(curr_slice).describe()

In [ ]:
dataset["Intersecting Street Name"].replace(np.NAN, pd.NA, inplace=True)

In [ ]:
curr_slice = next(ind_iter)
tmp_viewer(curr_slice).head()

In [ ]:
dataset["Property Damages"].unique()[0:10]

In [ ]:
dataset["Property Damages"].describe()

In [ ]:
li = ["Private Drive Flag"]
dataset.loc[:,li] = booleanize(dataset.loc[:,li], "Yes", "No")

In [ ]:
curr_slice = next(ind_iter)
tmp_viewer(curr_slice).head()

In [ ]:
tmp_viewer(curr_slice).dtypes

In [ ]:
s = ['Surface Condition','Traffic Control Type','Weather Condition','Person Helmet']
{c:dataset[c].unique() for c in s}

In [ ]:
dataset.head()

In [ ]:
uni_dict

In [ ]:
extract_file_name = lambda f: f.split(".")[0]
save_df_csv(dataset.to_csv(index=False, line_terminator="\n"),
file_name=f"{extract_file_name(csv_file_name)} revised.csv")

## Cleaning and Pre-process of the Data

In [ ]:
import pandas as pd
import numpy as np
from pprint import pprint
data_file_path = ""
csv_file_name = "Austin Bicycle Crashes 2010-2017 revised.csv"
dataset = pd.read_csv(data_file_path+csv_file_name)
dataset.head()

In [ ]:
dataset.drop_duplicates('Crash ID', inplace=True) # assuming Crash ID is the primary key
# Visualising missing value
import missingno as msno
msno.matrix(dataset)

In [ ]:
# another visualization of missing values based on counting
dataset.count().plot.box()

In [ ]:
# from previous boxplot, we decide drop the columns whose counts less then 2000
li = filter(lambda col_nm: dataset[col_nm].count() < 2000, dataset)
li = list(li)
li # the list of columns will be drop

In [ ]:
dataset.drop(li, axis=1, inplace=True)

In [ ]:
dataset.count(1)[0] # number of variables

In [ ]:
import missingno as msno
msno.matrix(dataset)

### Dropping Row

In [ ]:
msno.matrix(dataset)

In [ ]:
# we may assume that Crash ID is the primary key
# hence drop Case ID
dataset.drop("Case ID", axis=1, inplace = True) 
dataset = dataset.dropna()
msno.matrix(dataset)

We have reach at the point where there is no obvious missing data, thus the data is cleaner than initially.

In [ ]:
from pyproj import Proj, transform
# cartography(map making field), all maps are projected
# then we use Mercator projection here
lons, lats = [], []
inProj = Proj(init='epsg:3857')
outProj = Proj(init='epsg:4326')
for lon, lat in list(zip(dataset["Longitude"], dataset["Latitude"])):
    x, y = transform(outProj,inProj,lon,lat)
    lons.append(x)
    lats.append(y)
dataset["MercatorX"] = lons
dataset["MercatorY"] = lats

### Saving file

In [ ]:
def save_df_csv(csv_str, file_name, path=""):
	f = open(path+file_name, "w")
	f.write(csv_str)
	f.close()
extract_file_name = lambda f: f.split(".")[0]
save_df_csv(dataset.to_csv(index=False, line_terminator="\n"),
file_name=f"{extract_file_name(csv_file_name)} subset.csv")

## To discover outlier

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
import numpy as np
import pprint
%matplotlib inline

In [ ]:
url = "https://raw.githubusercontent.com/taimoon/data-wrangling-school-project/main/Austin%20Bicycle%20Crashes%202010-2017%20revised%20subset.csv"
df=pd.read_csv(url)
df.head()

In [ ]:
#create a list  for column indexing
outlier_col=["Crash Death Count","Crash Incapacitating Injury Count","Crash Non-incapacitating Injury Count",
             "Crash Not Injured Count","Crash Possible Injury Count","Crash Total Injury Count",
             "Crash Unknown Injury Count","Speed Limit" ]
print(len(outlier_col))

In [ ]:
#create a dict to store location of outlier
out_dict = {}
new_key=0
for b in outlier_col:
    z = np.abs(stats.zscore(df[b]))
    out_index= np.where(z>3)
    
    # add into dictionary
    new_key+=1
    out_dict[b] = out_index


pprint.pprint(out_dict)

In [ ]:
#to extract outlier value

for b in outlier_col:
    print("Outlier value for " + b)
    print(df[b].iloc[out_dict[b]])
    print(" ")
    print("-------------------------------------------------------------- ")

In [ ]:
#In the latest version of Pandas there is an easy way to do exactly this. 
#Column names (which are strings) can be sliced in whatever manner you like
df_measures= pd.DataFrame(df, columns=outlier_col)

In [ ]:
#Box Plot
for col in outlier_col:
    sns.boxplot( y=df[col] )
    plt.figure()   # plots figure for each iteration

In [ ]:
#Histogram
for col in outlier_col:
    sns.histplot(data=df[col])
    plt.figure()   # plots figure for each iteration

In [ ]:
x_col = "Speed Limit"
y_columns = ["Crash Death Count","Crash Incapacitating Injury Count","Crash Non-incapacitating Injury Count",
             "Crash Not Injured Count","Crash Possible Injury Count","Crash Total Injury Count",
             "Crash Unknown Injury Count"]

outlier_col=["Crash Death Count","Crash Incapacitating Injury Count","Crash Non-incapacitating Injury Count",
             "Crash Not Injured Count","Crash Possible Injury Count","Crash Total Injury Count",
             "Crash Unknown Injury Count","Speed Limit" ]
for y_col in y_columns:

    figure = plt.figure
    ax = plt.gca()
    ax.scatter(df[x_col], df[y_col])
    #ax.set_xlabel(x_col)
    #ax.set_ylabel(y_col)
    ax.set_title("{} vs {}".format(x_col, y_col))

    plt.legend()
    plt.show()

## Relationship Between Variables

In [ ]:
import pandas as pd
import seaborn as sb

In [ ]:
df = pd.read_csv("https://raw.githubusercontent.com/taimoon/data-wrangling-school-project/main/Austin%20Bicycle%20Crashes%202010-2017%20revised%20subset.csv")
df.head()

In [ ]:
# let we present by heat map

sb.heatmap(df.corr(),cmap="YlGnBu")

In the above particular heatmap with this color map, the dark color means that the correlation is very high. Our diagonal is of course very dark and then we can see that **between crash total injury count and Crash Incapacitating Injury Count** got also dark blue cube, which means that they are very correlative.

We also can see that between MercatorX and longitude are very correlative. It can also be seen that MercatorY has very high correlations with latitude. The combination between "logitude and latitude" and "MercatorY and MercatorX" are correlated together too.
We can explained that MercatorX, longitude, latitude and MercatorY are correlated each other is because they are geographic  measurement that related to each other, and often used and appear in combination. For example, (15°24'15"N, 30°10'3"E) for latitude and longitude. The MercatorY and MercatorX can be converted into latitude and longitude respectively.

In [ ]:
# let we just select particular row
list_selected=["Crash Total Injury Count", "Crash Unknown Injury Count", "Speed Limit"]
df_selected=pd.DataFrame(df, columns=list_selected)
df_selected.head()

In [ ]:
sb.heatmap(df_selected.corr(),cmap="RdBu_r", annot=True)

To zoom in more details, it is clearly show there are no correlation between Crash Total ijury Count, Crash unknown injury count and speed limit.

In [ ]:
sb.pairplot(df_selected)

Now, we will run a pairplot, which takes every two variables and shows us their scatter versus each other.

In [ ]:
# let create pairplot for crash total injury count and Crash Incapacitating Injury Count.

sb.pairplot(df,
    x_vars="Crash Total Injury Count",
    y_vars="Crash Incapacitating Injury Count",
    diag_kws={'bins':30}
)

When we go deep into the correlation between rash total injury count and Crash Incapacitating Injury Count, we cannot make statement that they have correlation. This is because they creates the plot without the regression line.